In [16]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
import json

In [17]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag",
    dimension=1536,
    metric="cosine",
    spec=(ServerlessSpec(cloud="aws", region="us-east-1"))
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '14b6270e9446dcb92c71fe4b2c610193', 'Date': 'Tue, 20 Aug 2024 21:12:42 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [31]:
data = json.load(open("reviews.json"))
data['reviews']

[{'id': '1',
  'professor': 'Dr. Alice Johnson',
  'school': 'University of California, Berkeley',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Dr. Johnson is an amazing professor. Her lectures are clear and concise, and she is always available to help outside of class.'},
 {'id': '2',
  'professor': 'Dr. Robert Smith',
  'school': 'Stanford University',
  'subject': 'Mathematics',
  'stars': 4,
  'review': 'Dr. Smith is very knowledgeable, but his lectures can be a bit fast-paced. Make sure to review the material ahead of time.'},
 {'id': '3',
  'professor': 'Dr. Emily Davis',
  'school': 'Massachusetts Institute of Technology',
  'subject': 'Physics',
  'stars': 5,
  'review': 'Dr. Davis makes complex topics easy to understand. Her enthusiasm for physics is contagious!'},
 {'id': '4',
  'professor': 'Dr. John Williams',
  'school': 'Harvard University',
  'subject': 'Chemistry',
  'stars': 3,
  'review': 'Dr. Williams knows his material, but his teaching style can be a

In [32]:
processed_data = []
client = OpenAI()
for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model='text-embedding-3-small'
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["id"],
        "metadata": {
            "professor":review['professor'],
            "school":review['school'],
            "review":review['review'],
            "subject":review['subject'],
            "stars":review['stars'],
            }
    })

In [34]:
processed_data[0]

{'values': [-0.018514624,
  -0.0067926263,
  0.0079424195,
  0.04693516,
  0.0064211544,
  0.00081296294,
  0.012229086,
  0.03219422,
  -0.0051829154,
  0.0047760652,
  0.034411255,
  0.00067366107,
  -0.0292932,
  0.017311763,
  -0.012453148,
  0.035401847,
  -0.024599684,
  0.0069518285,
  0.024859125,
  0.068114944,
  0.025731789,
  -0.0118458215,
  0.017547617,
  -0.03627451,
  -0.028255438,
  -0.036416024,
  0.00909811,
  0.0076888753,
  0.031203626,
  -0.0068162116,
  0.08646447,
  0.0054688896,
  -0.01948163,
  -0.026415769,
  -0.026439354,
  0.032147046,
  0.015059346,
  0.0039063497,
  0.0076417043,
  0.0070756525,
  -0.015472093,
  0.0038061112,
  -0.005651677,
  0.0019885532,
  0.026415769,
  -0.008708948,
  -0.0017880762,
  0.010648857,
  0.056463704,
  0.04603891,
  -0.028656391,
  -0.0006570775,
  0.02735919,
  -0.020059474,
  -0.033845205,
  -0.0025693462,
  0.018726893,
  0.037100002,
  0.018703308,
  -0.020790625,
  0.044505853,
  0.001863255,
  -0.023974668,
  -0.005

In [35]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
    )

{'upserted_count': 21}

In [36]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 21}},
 'total_vector_count': 21}